# **Homework**: Data talks club data engineering zoomcamp Data loading workshop

Hello folks, let's practice what we learned - Loading data with the best practices of data engineering.

Here are the exercises we will do




# 1. Use a generator

Remember the concept of generator? Let's practice using them to futher our understanding of how they work.

Let's define a generator and then run it as practice.

**Answer the following questions:**

- **Question 1: What is the sum of the outputs of the generator for limit = 5?**
- **Question 2: What is the 13th number yielded**

I suggest practicing these questions without GPT as the purpose is to further your learning.

In [10]:
def square_root_generator(limit):
    n = 1
    while n <= limit:
        yield n ** 0.5
        n += 1

# Example usage:
limit = 5
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)


1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979


In [11]:
1.0+1.4142135623730951+1.7320508075688772+2.0+2.23606797749979

8.382332347441762

In [12]:
limit = 13
generator = square_root_generator(limit)

for sqrt_value in generator:
    print(sqrt_value)

1.0
1.4142135623730951
1.7320508075688772
2.0
2.23606797749979
2.449489742783178
2.6457513110645907
2.8284271247461903
3.0
3.1622776601683795
3.3166247903554
3.4641016151377544
3.605551275463989


# 2. Append a generator to a table with existing data


Below you have 2 generators. You will be tasked to load them to duckdb and answer some questions from the data

1. Load the first generator and calculate the sum of ages of all people. Make sure to only load it once.
2. Append the second generator to the same table as the first.
3. **After correctly appending the data, calculate the sum of all ages of people.**




In [13]:
def people_1():
    for i in range(1, 6):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 25 + i, "City": "City_A"}

for person in people_1():
    print(person)


def people_2():
    for i in range(3, 9):
        yield {"ID": i, "Name": f"Person_{i}", "Age": 30 + i, "City": "City_B", "Occupation": f"Job_{i}"}


for person in people_2():
    print(person)


{'ID': 1, 'Name': 'Person_1', 'Age': 26, 'City': 'City_A'}
{'ID': 2, 'Name': 'Person_2', 'Age': 27, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 28, 'City': 'City_A'}
{'ID': 4, 'Name': 'Person_4', 'Age': 29, 'City': 'City_A'}
{'ID': 5, 'Name': 'Person_5', 'Age': 30, 'City': 'City_A'}
{'ID': 3, 'Name': 'Person_3', 'Age': 33, 'City': 'City_B', 'Occupation': 'Job_3'}
{'ID': 4, 'Name': 'Person_4', 'Age': 34, 'City': 'City_B', 'Occupation': 'Job_4'}
{'ID': 5, 'Name': 'Person_5', 'Age': 35, 'City': 'City_B', 'Occupation': 'Job_5'}
{'ID': 6, 'Name': 'Person_6', 'Age': 36, 'City': 'City_B', 'Occupation': 'Job_6'}
{'ID': 7, 'Name': 'Person_7', 'Age': 37, 'City': 'City_B', 'Occupation': 'Job_7'}
{'ID': 8, 'Name': 'Person_8', 'Age': 38, 'City': 'City_B', 'Occupation': 'Job_8'}


In [14]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people",
										write_disposition="append")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:///c:\Users\oheld\de-zoomcamp-24\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708281988.984497 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.19 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:///c:\Users\oheld\de-zoomcamp-24\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708281989.624188 is LOADED and contains no failed jobs


In [15]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people").df()
display(peoples)

print("\n\n\n sum of age below:")

age = conn.sql("SELECT SUM(age) FROM people").df()
display(age)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_table        │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708281988.984497,rpSFzlg4kBKDAg,None
1,2,Person_2,27,City_A,1708281988.984497,83k0+VyB3p3rXA,None
2,3,Person_3,28,City_A,1708281988.984497,vNfnNkeaNKXVLw,None
3,4,Person_4,29,City_A,1708281988.984497,naHj0psbQ8YYmA,None
4,5,Person_5,30,City_A,1708281988.984497,gCBXZxo5cTuceg,None
5,3,Person_3,33,City_B,1708281989.624188,71/WAIOo+FyXow,Job_3
6,4,Person_4,34,City_B,1708281989.624188,uUn7hRQkf6q5Zw,Job_4
7,5,Person_5,35,City_B,1708281989.624188,DR+RZPdrnYfzJQ,Job_5
8,6,Person_6,36,City_B,1708281989.624188,Eve+UHI1N19bbA,Job_6
9,7,Person_7,37,City_B,1708281989.624188,OWO8AmlwZa1paA,Job_7





 sum of age below:


,sum(age)
0,353.0


# 3. Merge a generator

Re-use the generators from Exercise 2.

A table's primary key needs to be created from the start, so load your data to a new table with primary key ID.

Load your first generator first, and then load the second one with merge. Since they have overlapping IDs, some of the records from the first load should be replaced by the ones from the second load.

After loading, you should have a total of 8 records, and ID 3 should have age 33.

Question: **Calculate the sum of ages of all the people loaded as described above.**


In [16]:
import dlt

# define the connection to load to.
# We now use duckdb, but you can switch to Bigquery later
generators_pipeline = dlt.pipeline(destination='duckdb', dataset_name='people_data')


# we can load any generator to a table at the pipeline destnation as follows:
info = generators_pipeline.run(people_1(),
										table_name="people_table",
										write_disposition="replace")

# the outcome metadata is returned by the load and we can inspect it by printing it.
print(info)

# we can load the next generator to the same or to a different table.
info = generators_pipeline.run(people_2(),
										table_name="people_table",
										write_disposition="merge",
                                        primary_key="id")

print(info)

Pipeline dlt_ipykernel_launcher load step completed in 0.20 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:///c:\Users\oheld\de-zoomcamp-24\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708281990.3718975 is LOADED and contains no failed jobs
Pipeline dlt_ipykernel_launcher load step completed in 0.36 seconds
1 load package(s) were loaded to destination duckdb and into dataset people_data
The duckdb destination used duckdb:///c:\Users\oheld\de-zoomcamp-24\workshops\dlt_resources\dlt_ipykernel_launcher.duckdb location to store data
Load package 1708281991.0489154 is LOADED and contains no failed jobs


In [17]:
# show outcome

import duckdb

conn = duckdb.connect(f"{generators_pipeline.pipeline_name}.duckdb")

# let's see the tables
conn.sql(f"SET search_path = '{generators_pipeline.dataset_name}'")
print('Loaded tables: ')
display(conn.sql("show tables"))

# and the data

print("\n\n\n people table below:")

peoples = conn.sql("SELECT * FROM people_table").df()
display(peoples)

print("\n\n\n sum of age below:")

age = conn.sql("SELECT SUM(age) FROM people_table").df()
display(age)

# As you can see, the same data was loaded in both cases.

Loaded tables: 


┌─────────────────────┐
│        name         │
│       varchar       │
├─────────────────────┤
│ _dlt_loads          │
│ _dlt_pipeline_state │
│ _dlt_version        │
│ people              │
│ people_table        │
└─────────────────────┘




 people table below:


,id,name,age,city,_dlt_load_id,_dlt_id,occupation
0,1,Person_1,26,City_A,1708281990.3718975,OIBxkbxCXZ7mFw,None
1,2,Person_2,27,City_A,1708281990.3718975,ITU6ZaN9jqQypw,None
2,3,Person_3,33,City_B,1708281991.0489154,BIxY1StkaT/KuQ,Job_3
3,6,Person_6,36,City_B,1708281991.0489154,f5k5TmTm8xzexA,Job_6
4,8,Person_8,38,City_B,1708281991.0489154,8iE7i08E74biKg,Job_8
5,7,Person_7,37,City_B,1708281991.0489154,aJsHYRl2T3jtOA,Job_7
6,4,Person_4,34,City_B,1708281991.0489154,KKcM815l6t2zIg,Job_4
7,5,Person_5,35,City_B,1708281991.0489154,agg/sPFA1S8uMQ,Job_5





 sum of age below:


,sum(age)
0,266.0
